In [1]:
import pandas as pd

import tweepy

import config

from bots.twitter import TwitterBot
from helpers.data_helpers import save_to_parquet, clean_text, clean_tweets_df, save_to_text

2023-03-27 16:42:54,095 - INFO     | config     | Loading environment variables
2023-03-27 16:42:54,096 - INFO     | config     | Directory raw_data already exists
2023-03-27 16:42:54,096 - INFO     | config     | Directory data already exists


# Extracting Tweets

In [2]:
# initialize twitter bot
twitter_bot = TwitterBot(bearer=config.TWTR_BEARER_TOKEN,
                         api=config.TWTR_API,
                         api_secret=config.TWTR_API_SECRET,
                         access=config.TWTR_ACCESS_TOKEN,
                         access_secret=config.TWTR_ACCESS_TOKEN_SECRET)

In [3]:
apparel = [
    'context:46.783337567798169600 -is:retweet lang:en',    # context: products
    'context:67.839543390668673024 -is:retweet lang:en',    # context: apparel
    'context:131.1248316002317643776 -is:retweet lang:en',  # context: athletic apparel
    'context:65.1256236649253449729 -is:retweet lang:en',   # context: fashion and beauty
    'context:131.1095391406816784384 -is:retweet lang:en',  # context: shopping
    'context:67.1486758812849635329 -is:retweet lang:en',   # context: retail sales
    'context:131.1407812892909473795 -is:retweet lang:en',  # context: ecommerce
    'context:165.1407812892909473795 -is:retweet lang:en',  # context: ecommerce
]

In [43]:
cars = [
    #'context:46.781974597251964928 -is:retweet lang:en',    # context: car rental/sharing
    #'context:45.781972125171060736 -is:retweet lang:en',     # context: auto
    'context:46.781972125179518977 -is:retweet lang:en',     # context: auto manufacturer
    #'context:45.1196845866138533888 -is:retweet lang:en',     # context: automobile brands
    #'context:131.1196845866138533888 -is:retweet lang:en',     # context: automobile brands
]

In [44]:
dfs = []
for query in cars:
    tweets = twitter_bot.get_recent_tweets(query=query, limit=5000)
    df = pd.DataFrame.from_dict(tweets)
    print(len(df))
    dfs.append(df)

TooManyRequests: 429 Too Many Requests
Too Many Requests

In [35]:
tweets = pd.concat(dfs, ignore_index=True)
tweets = tweets.loc[~tweets['id'].duplicated()].copy()

In [36]:
tweets

,id,created_at,text,entities
0,1640369503325540358,2023-03-27 15:05:40+00:00,"Also, @DoorDash and @UberEats Explain the fu...","{Uber - Super Bowl, Food & Beverage Business, ..."
1,1640369500699885574,2023-03-27 15:05:40+00:00,"Not cool @Uber , waiting until 15 min prior to...","{Transportation, Uber, Travel & Transportation..."
2,1640369492365660164,2023-03-27 15:05:38+00:00,@AbjaFCB Stfu you so dumb... Ronaldo gets hat-...,"{Soccer, Food & Beverage Business, Travel & Tr..."
3,1640369461566873603,2023-03-27 15:05:30+00:00,@Uber_Support 08129270858\nalohanprecious11@gm...,"{Transportation, Uber, Travel & Transportation..."
4,1640369453476073473,2023-03-27 15:05:28+00:00,Shoutout to uber drivers that dont make conver...,"{Transportation, Uber, Travel & Transportation..."
...,...,...,...,...
4995,1640109319810105345,2023-03-26 21:51:48+00:00,@RegalSmiles We understand that driver was rud...,"{Transportation, Uber, Travel & Transportation..."
4996,1640109319495512065,2023-03-26 21:51:48+00:00,Accidentally got my Uber eats order for free. ...,"{Uber - Super Bowl, Food & Beverage Business, ..."
4997,1640109301019684866,2023-03-26 21:51:43+00:00,On that note everyone please read this article...,"{Travel & Transportation Business, Airbnb, Ube..."
4998,1640109279431606272,2023-03-26 21:51:38+00:00,@Uber Perhaps you can virtue signal after you ...,"{Transportation, Uber, Travel & Transportation..."


In [19]:
save_to_parquet(data_dir=config.RAW_DATA_DIR, df=tweets, name='tweets_apparel_32k')

# Cleaning Tweets

In [20]:
tweets_df = pd.read_parquet(f'{config.RAW_DATA_DIR}/tweets_apparel_32k_2023327.parquet')

In [21]:
tweets_df['created_at'] = pd.to_datetime(tweets_df['created_at'].dt.strftime('%Y-%m-%d'))

In [22]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32398 entries, 0 to 32397
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          32398 non-null  int64         
 1   created_at  32398 non-null  datetime64[ns]
 2   text        32398 non-null  object        
 3   entities    32398 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1012.6+ KB


In [23]:
tweets_df['text'] = tweets_df['text'].map(lambda x: clean_text(x))

In [24]:
df_clean = clean_tweets_df(df=tweets_df, text_col='text', n=2)

In [25]:
df_clean

,id,created_at,entities,full_text
0,1640367128854564864,2023-03-27,"[Entertainment & Leisure Business, B2B, Baseba...",\nTWEET: hahdhdh thats why he is practicing i ...
1,1640367079076376579,2023-03-27,"[B2B, Services - B2B, Technology Business, Pro...",\nTWEET: That's right - We're using #AI to hel...
2,1640366751463374853,2023-03-27,"[Hasbro, Products - B2B, B2B]",\nTWEET: Indy Goes trick or treating as Two Fa...
3,1640366569673850881,2023-03-27,"[B2B, Services - B2B, Technology Business, Dat...",\nTWEET: Forget the DIY headaches when you cho...
4,1640366205818003462,2023-03-27,"[Samsung , NVIDIA, B2B, AMD, Samsung USA, Sams...",\nTWEET: The global semiconductor supply chain...
...,...,...,...,...
32114,1640356360557588480,2023-03-27,"[Online - Retail, eCommerce industry, Motorcyc...",\nTWEET: Check out Harley Davidson Mens Hat Ca...
32115,1640356357537959938,2023-03-27,"[Textile, Apparel & Luxury Good Business, Onli...",\nTWEET: Check out this listing I just added t...
32116,1640356347308036098,2023-03-27,"[Samsung , Technology Business, eCommerce indu...",\nTWEET: Samsung Galaxy S21 FE Warna *Graphit...
32117,1640356342249713666,2023-03-27,"[Online - Retail, eCommerce industry, Industri...",\nTWEET: Check out this listing I just added t...


In [26]:
save_to_text(df=df_clean, col='full_text', out_dir=config.DATA_DIR, name='tweets_apparel_32k_2023327')

2023-03-27 17:03:12,622 - INFO     | helpers.data_helpers | Saved corpus to .txt file: tweets_apparel_32k_2023327.txt
